In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from openpyxl import load_workbook

In [ ]:
filename = "search_keys"
company = Company.objects.get(name="PRE-UHE-ITA")

In [ ]:
wb = load_workbook('./' + filename + ".xlsx")
sheetname = wb.sheetnames[0]
ws = wb[sheetname]

In [ ]:
header = []
values = []
good_fields = []
bad_fields = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

In [ ]:
values[0]

In [ ]:
tags = []
for value in values:
    for level, name in value.items():
        if not name or name == "NAO":
            continue
        tag = {
            "name": name,
            "level": level,
            "parent_tags": [{
                "name": b,
                "level": a
            } for a, b in [c for c in value.items() if c[0] < level and c[1] and c[1] != "NAO"]]
        }
        tags.append(tag)

In [ ]:
tags

In [ ]:
unique_tags = []
for tag in tags:
    name = tag['name']
    level = tag['level']
    parent_tags = tag['parent_tags']
    existing_tag = None
    try:
        existing_tag = next(a for a in unique_tags if a['name'] == name and a['level'] == level)
    except Exception:
        unique_tags.append(tag)
    else:
        existing_tag_index = unique_tags.index(existing_tag)
        unique_tags[existing_tag_index]['parent_tags'] += parent_tags

In [ ]:
unique_tags

In [ ]:
for tag in unique_tags:
    new_parent_tags = []
    for index, parent in enumerate(tag['parent_tags']):
        if tag['parent_tags'].index(parent) == index:
            new_parent_tags.append(parent)
    tag['parent_tags'] = new_parent_tags

In [ ]:
sorted_tags = sorted(unique_tags, key=lambda x: x['level'])

In [ ]:
sorted_tags

In [ ]:
SearchTag.objects.all().delete()

In [ ]:
for tag in sorted_tags:
    tag_obj = SearchTag(
        company = company,
        name = tag['name'],
        level = tag['level'],
        kind = 'text_field' if tag['name'] == "ABERTO" else 'select_option',
    )
    tag_obj.save()
    parent_objs = []
    for parent in tag['parent_tags']:
        parent_obj = SearchTag.objects.get(name=parent['name'], level=parent['level'])
        parent_objs.append(parent_obj)
    tag_obj.parent_tags.set(parent_objs)
#         print(tag_obj, parent_obj)
#         tag_obj.parent_tags.add(parent_obj)
#         print(tag_obj.level, tag_obj.name)
#         print('added level {} parent to level {} object'.format(parent_obj.level, tag_obj.level))

In [ ]:
for tag in SearchTag.objects.all().order_by('level'):
    print(tag.level, tag.parent_tags.count())

In [ ]:
formularios = {
	"Requisições": [{
		"name": "Requisição",
		"level": 1
	}],
	"Ocorrências ambientais": [{
		"name": "Ocorrência",
		"level": 1
	}, {
		"name": "Ambiental",
		"level": 3
	}],
	"Ocorrências patrimoniais": [{
		"name": "Ocorrência",
		"level": 1
	}, {
		"name": "Patrimonial",
		"level": 3
	}],
	"Ocorrência natureza humana e material": [{
		"name": "Ocorrência",
		"level": 1
	}, {
		"name": "Humana e material",
		"level": 3
	}],
	"Diálogo com a sociedade": [{
		"name": "Comunicação",
		"level": 1
	}, {
		"name": "Diálogo com a sociedade",
		"level": 2
	}],
	"Visita técnica a parte interessada": [{
		"name": "Comunicação",
		"level": 1
	}, {
		"name": "Visita técnica a parte interessada",
		"level": 2
	}],
	"Vistorias técnicas": [{
		"name": "Relatório",
		"level": 1
	}, {
		"name": "Vistoria técnica",
		"level": 2
	}],
	"Resumo Executivo": [{
		"name": "Relatório",
		"level": 1
	}, {
		"name": "Resumo executivo",
		"level": 2
	}],
	"Verificação de serviços": [{
		"name": "Relatório",
		"level": 1
	}, {
		"name": "Verificação de serviços",
		"level": 2
	}]
}

In [ ]:
for form, tags in formularios.items():
#     print(form)
    o_type = OccurrenceType.objects.filter(company=company, name=form).first()
    relation = SearchTagOccurrenceType(
        company = company,
        occurrence_type = o_type
    )
    relation.save()
    for tag in tags:
        s_tag = SearchTag.objects.get(company=company, name=tag['name'], level=tag['level'])
        relation.search_tags.add(s_tag)

## Adicionando Description

In [ ]:
description = {
    "Requisição": "Qualquer tipo de solicitação",
    "Ocorrência": "Um acontecimento fora do ordinário",
    "Comunicação": "Diálogos com a sociedade ou visitas a partes interessadas",
    "Relatório": "Vistorias técnicas, resumos executivos ou verificações de serviço",
    "Controle Operacional": "Controle de assuntos recorrentes sem periodicidade definida",
    "Monitoramentos": "Controles periódicos e resultados de análises"
}

In [ ]:
for key, value in description.items():
    search_tags = SearchTag.objects.filter(level=1, company=company, name=key)
    if search_tags.exists():
        search_tags.update(description=value)